# Code to generate municipality forest polygons for Colombia

### Load packages

In [1]:
import os
import ee
import geemap

In [2]:
ee.Initialize()
#ee.Authenticate()

### Set data parameters

In [3]:
# Selected country 
country = 'Colombia';
# Canopy cover percentage (e.g. 10%).
cc = ee.Number(25);
# Minimum forest area in pixels (e.g. 6 pixels, ~ 0.5 ha in this example).
pixels = ee.Number(6);
# Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = ee.Number(6);

### Import forest cover data

In [4]:
#Load country features from Large Scale International Boundary (LSIB) dataset.
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');
country_shp = countries.filter(ee.Filter.eq('country_na', ee.String(country)));

# Draw any shapes on the map using the Drawing tools before executing this code block
#roi = Map.user_roi
roi = ee.Geometry.Polygon([[[-79, -4.3], [-79, 12.55], [-66.5, 12.55], [-66.5, -4.3]]])
print(roi.getInfo())
#Map.addLayer(roi, {}, 'polygonGeoJSON')
#country_shp=roi

#Load GFC data for 2020
gfc = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')

{'type': 'Polygon', 'coordinates': [[[-79, -4.3], [-66.5, -4.3], [-66.5, 12.55], [-79, 12.55], [-79, -4.3]]]}


In [5]:
#Select tree cover
canopyCover = gfc.select(['treecover2000']) #Make areas below the canopy cover transparent
canopyCover = canopyCover.gte(cc).selfMask()
canopyCover = canopyCover.connectedPixelCount().gte(pixels).selfMask().clip(country_shp)

### Map baseline tree cover (2000)

In [6]:
Map = geemap.Map()
Map

No such comm: fb1a0d12d2b74b7ea4496d0f41ab1f1f


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [7]:
treecover = canopyCover
Map.addLayer(treecover, {'min': 0,'max': 1,  'palette': ['green']} ,'cover2000')
Map.setCenter(-68, 5, 6)

### Load population data form Grided population of the world - NASA 

In [8]:
#Load population data and mask for forest area
#gpw = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Density").filter(ee.Filter.date('2015-01-01', '2016-01-01'))
gpw2005 = ee.Image('CIESIN/GPWv4/population-count/2005').updateMask(treecover.eq(1))
gpw2010 = ee.Image('CIESIN/GPWv4/population-count/2010').updateMask(treecover.eq(1))
gpw2015 = ee.Image('CIESIN/GPWv4/population-count/2015').updateMask(treecover.eq(1))
gpw2020 = ee.Image('CIESIN/GPWv4/population-count/2020').updateMask(treecover.eq(1))

In [9]:
#Plot data of poulation living on forests
#print(geemap.image_props(gpw2005).getInfo())

raster_vis = {
  "max": 1000.0,
  "palette": [
    "ffffe7",
    "86a192",
    "509791",
    "307296",
    "2c4484",
    "000066"
  ],
  "min": 0.0
}

Map.addLayer(gpw2020, raster_vis, 'population_2020')

{'IMAGE_DATE': '2005-01-01', 'NOMINAL_SCALE': 927.662423277286, 'system:asset_size': '305.637526 MB', 'system:band_names': ['population-count'], 'system:id': 'CIESIN/GPWv4/population-count/2005', 'system:index': '2005', 'system:time_end': '2005-01-01 00:00:00', 'system:time_start': '2005-01-01 00:00:00', 'system:version': 1493666437460000.0}


## Population living in forests by municipalities

### Import municipality shapefile

In [10]:
#geemap.ee_search()
municipios = ee.FeatureCollection('users/emileg1079/municipios')

### Loop for counting popultion

In [ ]:
#Define function to aggregate raster data over a geometry

def mun_temp(region,var,agg_fun) :
    total = var.reduceRegion(
        reducer=agg_fun,
        geometry=region.geometry(),
        scale=1000,
        maxPixels=1e10
        );
    return region.set(total)

In [11]:
#Loop that sums population over 1km2 pixels of forests

for y in ["2005","2010","2015","2020"] :
    gpw_y = ee.Image(f'CIESIN/GPWv4/population-count/{y}').updateMask(treecover.eq(1))
    gpw_y_pop = gpw_y.select("population-count")
    gpw_y_sum = municipios.map(lambda x: mun_temp(x, gpw_y_pop, ee.Reducer.sum()))
    task = ee.batch.Export.table.toDrive(collection=gpw_y_sum,
                                     folder="gee_export_test", # Nombre de la carpeta
                                     description=f"population_forest_{y}") # Nombre del archivo
                                     #fileFormat="shp") # Tipo de archivo
    task.start()

In [12]:
# EJEMPLO PARA UN SOLO AÑO
# Se seleccionan las variables que se quieren extraer
gpw2015_pop = gpw2015.select("population-count")
# Con la función "mun_temp" se aplica la reducción en el shape file deseado
# "municipios" es un featureCollection con los polígonos de los municipios
gpw2015_sum = municipios.map(lambda x: mun_temp(x, gpw2015_pop, ee.Reducer.sum()))

In [13]:
task = ee.batch.Export.table.toDrive(
    collection=gpw2015_sum,
    folder="gee_export_test", # Nombre de la carpeta
    description="gpw2015") # Nombre del archivo                 
#    fileFormat="shp") # Tipo de archivo
task.start()

#print(gpw2015_sum)